In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

#from subprocess import check_output
#print(check_output(["ls", "./input"]).decode("utf8"))
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

**GPU에서 실행** : CPU와 관련하여 일부 호환성 문제가 있습니다.
1. 딥러닝에는 하이퍼파라미터가 많아서, 튜닝하는데 몇 주 또는 몇 달이 걸립니다. 일반적으로 연구자들은 튜닝을 하고, 그들이 다른 것들보다 더 나은 성능을 가진 아키텍처를 발견했을 때 논문을 발표합니다.

2. 이 모델은 pre-trained 모델이기 때문에 매우 빠르게 수렴되었지만 그래도 이 모델을 사용하려면 GPU가 필요합니다. 일부 라이브러리 문제로 인해 CPU에서는 작동하지 않습니다.

3. 우리의 목적을 위해, 우리는 연구자들에 의해서 이용할 수 있도록 만들어진 아키텍처들을 사용할 수 있습니다.

4. 이미 피처를 추출할 방법을 알고 있는 레이어인 pre-trained nets 를 이용하면 우리는 하이퍼파라미터들을 튜닝할 필요가 없습니다. 이미 일부 데이터셋(예:imagenet)에 대해 train되었으므로, pre-trained weight는 적절한 weight의 초기화를 제공하고, 이로 인해 우리의 Convnet은 딥 아키텍처에 매우 빠르게 수렴합니다. 이것이 **Transfer Learning**에 대한 아이디어입니다. 예로는 VGG16, inceptionNet, googlenet, Resnet 등이 있습니다.

5. 이 커널에서는 크기가 작은 이미지에 대해서 잘 작동하는 pre-trained VGG-16 network를 사용할 것입니다.

6.** VGG 아키텍처는 크기가 작은 이미지(CIFAR-10)에서 잘 작동하는 것으로 확인되었습니다. ** 따라서 이 데이터셋에도 잘 작동할 것으로 예상했습니다.

1. 코드에는 data augmentation 단계도 포함되어 있으므로 성능이 크게 향상됩니다.

2. **GPU가 필요합니다**

관심이 있는 분은 여기의 논문 링크를 참고해주세요.
[https://arxiv.org/pdf/1409.1556.pdf](http://)

keras 라이브러리 설명서입니다.
[https://keras.io/applications/#vgg16](http://)




In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab


plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

In [ ]:
train = pd.read_json("../input/train.json")
target_train=train['is_iceberg']
test = pd.read_json("../input/test.json")




Keras 는 pretrained VGG 실행을 제공합니다. 즉, 라이브러리에 있으므로 우리가 만들 필요가 없습니다.
여기서 우리는 VGG의 마지막 레이어를 제거하고 바이너리 예측을 위한 시그모이드 레이어를 넣습니다.

다음 코드는 모델의 weight가 다운로드 되지 않았기 때문에 kaggle 노트북에서 작동하지 않습니다. 그러나 본인의 노트북에 코드를 복붙하면 사용할 수 있습니다.

In [ ]:

target_train=train['is_iceberg']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce') #inc_angle에서 na 값을 NAN으로 바꿔줌
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')#133개의 NA가 있음
train['inc_angle']=train['inc_angle'].fillna(method='pad') #결측값을 앞 방향부터 채워나감
X_angle=train['inc_angle']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle=test['inc_angle']

#Generate the training data
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_band_3=(X_band_1+X_band_2)/2 #band_1과 band_2 각 행렬의 값을 더한 평균값으로 만듦. 그래서 band_1과 bnad_2의 크기가 같아야됨.
#X_band_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in train["inc_angle"]])

#X_band_1[:, :, :, np.newaxis] -> 같은 배열에 대해 차원을 1차원 증가
# axis=-1 설정 -> 맨 마지막 차원 내에서 합쳐짐
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis]
                          , X_band_2[:, :, :, np.newaxis]
                         , X_band_3[:, :, :, np.newaxis]], axis=-1)



X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_band_test_3=(X_band_test_1+X_band_test_2)/2
#X_band_test_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in test["inc_angle"]])
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis]
                          , X_band_test_2[:, :, :, np.newaxis]
                         , X_band_test_3[:, :, :, np.newaxis]], axis=-1)

#Import Keras.
from matplotlib import pyplot
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.optimizers import rmsprop
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input	

#Data Aug for multi-input
from keras.preprocessing.image import ImageDataGenerator
batch_size=64
# image transformations 정의
gen = ImageDataGenerator(horizontal_flip = True, #수평 방향으로 뒤집기
                         vertical_flip = True, #수직 방향으로 뒤집기
                         width_shift_range = 0., # 지정된 수평방향 이동 범위 내에서 임의로 원본이미지를 이동
                         height_shift_range = 0., # 지정된 수직방향 이동 범위 내에서 임의로 원본이미지를 이동
                         channel_shift_range=0, # 임의 채널 이동을 위한 범위
                         zoom_range = 0.2, # 지정된 확대/축소 범위내에서 임의로 원본이미지를 확대/축소 -> 0.8배에서 1.2배 크기 변화 시킴
                         rotation_range = 10) # 지정된 각도 범위내에서 임의로 원본이미지를 회전

# 두 generators를 합치는 함수입니다
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1]], X1i[1]

# Finally create generator
def get_callbacks(filepath, patience=2):
   es = EarlyStopping('val_loss', patience=10, mode="min")
   msave = ModelCheckpoint(filepath, save_best_only=True)
   return [es, msave]


def getVggAngleModel():
    input_2 = Input(shape=[1], name="angle")
    angle_layer = Dense(1, )(input_2)
    base_model = VGG16(weights='imagenet', include_top=False, 
                 input_shape=X_train.shape[1:], classes=1)
    x = base_model.get_layer('block5_pool').output
    

    x = GlobalMaxPooling2D()(x)
    merge_one = concatenate([x, angle_layer])
    merge_one = Dense(512, activation='relu', name='fc2')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    merge_one = Dense(512, activation='relu', name='fc3')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    
    predictions = Dense(1, activation='sigmoid')(merge_one)
    
    model = Model(input=[base_model.input, input_2], output=predictions)
    
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    return model


#Using K-fold Cross Validation with Data Augmentation.
def myAngleCV(X_train, X_angle, X_test):
    K=3
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout= target_train[test_idx]
        
        #Angle
        X_angle_cv=X_angle[train_idx]
        X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
        file_path = "%s_aug_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=5)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
        galaxyModel= getVggAngleModel()
        galaxyModel.fit_generator(
                gen_flow,
                steps_per_epoch=24,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data=([X_holdout,X_angle_hold], Y_holdout),
                callbacks=callbacks)

        #Getting the Best Model
        galaxyModel.load_weights(filepath=file_path)
        #Getting Training Score
        score = galaxyModel.evaluate([X_train_cv,X_angle_cv], y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = galaxyModel.evaluate([X_holdout,X_angle_hold], Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #Getting validation Score.
        pred_valid=galaxyModel.predict([X_holdout,X_angle_hold])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=galaxyModel.predict([X_test, X_test_angle])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=galaxyModel.predict([X_train, X_angle])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log




In [ ]:
preds=myAngleCV(X_train, X_angle, X_test)

In [ ]:
#Submission for each day.
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=preds
submission.to_csv('sub.csv', index=False)